<a href="https://colab.research.google.com/github/some-kim/Temp-Hack-MIT/blob/main/HackMIT_25'.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Propsperty Path

This is the output of a website designed to help individuals achieve their financial goals in a more tailored way. Instead of providing generic advice, the platform begins with a guided survey that gathers detailed information about each person’s goals, priorities, and current financial situation. We prompt them with a survery which converts their data into a dataset, which is the file uploaded in this program, and uses their data to build graphs to display where they stand now and if they follow our guidance, it displays where they will be in the future and when they will achieve their financial goals.

In [58]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib as font_manager
from matplotlib.patches import Wedge, Rectangle
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.express as px
from matplotlib.patches import Wedge
import matplotlib.patches as mpatches
from datetime import datetime, timedelta
import warnings
import math

In [51]:
UserData = pd.read_csv ("/content/sarah_martinez_financial_survey.csv")

In [69]:
# Sample financial data (replace with your actual data)
sarah_data = {
    'salary': 85000 / 12, 'freelance': 8000 / 12, 'dividends': 1200 / 12,
    'rent': -1800, 'utilities': -(120 + 45 + 35 + 65 + 35),
    'insurance': -(220 + 95 + 65), 'debt_payments': -(425 + 180 + 295),
    'phone': -85, 'gym': -65, 'subscriptions': -25,
    'groceries': -(110 * 4.33), 'dining_out': -(30 * 4.33),
    'gas_transport': -(45 * 4.33),
}

# Calculate totals
income_amounts = [sarah_data['salary'], sarah_data['freelance'], sarah_data['dividends']]
total_income = sum(income_amounts)
total_expenses = sum([sarah_data['rent'] + sarah_data['utilities'], sarah_data['insurance'],
                     sarah_data['debt_payments'], sarah_data['groceries'] + sarah_data['dining_out'],
                     sarah_data['gas_transport'], sarah_data['phone'] + sarah_data['gym'] + sarah_data['subscriptions']])
net_cash_flow = total_income + total_expenses

# CALCULATE FINANCIAL GROWTH GRADE
def calculate_financial_grade(income, expenses, net_flow):
    savings_rate = (net_flow / income) * 100 if income > 0 else 0

    grade = 0

    # Savings rate scoring (40% weight)
    if savings_rate >= 30:
        grade += 40
    elif savings_rate >= 20:
        grade += 32
    elif savings_rate >= 15:
        grade += 24
    elif savings_rate >= 10:
        grade += 16
    else:
        grade += savings_rate * 1.6

    # Emergency fund potential (20% weight)
    emergency_fund_months = net_flow / (abs(expenses) / 12) if expenses != 0 else 0
    if emergency_fund_months >= 6:
        grade += 20
    elif emergency_fund_months >= 3:
        grade += 15
    elif emergency_fund_months >= 1:
        grade += 10
    else:
        grade += emergency_fund_months * 10

    # Debt-to-income ratio (20% weight)
    debt_payments = abs(sarah_data['debt_payments'])
    debt_ratio = (debt_payments / income) * 100
    if debt_ratio <= 10:
        grade += 20
    elif debt_ratio <= 20:
        grade += 15
    elif debt_ratio <= 30:
        grade += 10
    else:
        grade += max(0, 20 - (debt_ratio - 10))

    # Diversified income (20% weight)
    primary_income = max(income_amounts)
    income_diversity = ((income - primary_income) / income) * 100
    if income_diversity >= 20:
        grade += 20
    elif income_diversity >= 10:
        grade += 15
    else:
        grade += income_diversity * 1.5

    return min(100, max(0, grade))

# DETERMINE COLOR BASED ON GRADE
def get_grade_color(grade):
    if grade >= 80:
        return '#27AE60'  # Green - Excellent
    elif grade >= 60:
        return '#F1C40F'  # Yellow - Good
    else:
        return '#E74C3C'  # Red - Needs Improvement

# GENERATE DETAILED FEEDBACK FOR HOVER
def get_grade_feedback(grade, savings_rate, net_flow, total_income):
    debt_ratio = (abs(sarah_data['debt_payments']) / total_income) * 100

    feedback = f"Financial Health Score: {grade:.1f}/100<br><br>"

    if grade >= 80:
        feedback += "🌟 Excellent financial health! You're doing great.<br>"
    elif grade >= 60:
        feedback += "👍 Good financial health with room for improvement.<br>"
    else:
        feedback += "⚠️ Needs attention to improve financial stability.<br>"

    feedback += f"<br><b>Detailed Breakdown:</b><br>"
    feedback += f"• Savings Rate: {savings_rate:.1f}% (Target: >20%)<br>"
    feedback += f"• Monthly Surplus: ${net_flow:,.0f}<br>"
    feedback += f"• Debt-to-Income: {debt_ratio:.1f}% (Target: <20%)<br>"
    feedback += f"• Income Sources: Multiple streams ✓<br><br>"

    feedback += f"<b>Grade Components:</b><br>"
    feedback += f"• Savings Performance: 40% weight<br>"
    feedback += f"• Emergency Fund Potential: 20% weight<br>"
    feedback += f"• Debt Management: 20% weight<br>"
    feedback += f"• Income Diversification: 20% weight"

    return feedback

# Calculate the grade
grade = calculate_financial_grade(total_income, total_expenses, net_cash_flow)
grade_color = get_grade_color(grade)
remaining_percentage = 100 - grade
savings_rate = (net_cash_flow / total_income) * 100
feedback_text = get_grade_feedback(grade, savings_rate, net_cash_flow, total_income)

# CREATE THE INTERACTIVE PIE CHART
fig = go.Figure()

# Add the pie chart
fig.add_trace(go.Pie(
    labels=['Financial Health Score', 'Room for Growth'],
    values=[grade, remaining_percentage],
    hole=0.6,  # Creates donut chart
    marker_colors=[grade_color, '#2C3E50'],  # Dynamic color based on grade + dark grey
    textinfo='none',  # Hide default text
    hovertemplate='<b>%{label}</b><br>' +
                  'Percentage: %{value:.1f}%<br>' +
                  f'<br>{feedback_text}<extra></extra>',
    showlegend=False,
    name="Financial Grade"
))

# Update layout for professional appearance
fig.update_layout(
    title=dict(
        text=f'🎯 Financial Growth Grade<br><sub>Interactive assessment of your financial health</sub>',
        font=dict(size=20, family='Times New Roman', color='#2C3E50'),
        x=0.5
    ),
    height=500,
    width=500,
    plot_bgcolor='white',
    paper_bgcolor='#F8F9FA',
    font=dict(family='Times New Roman', size=12),
    annotations=[
        # Grade percentage in center of donut
        dict(
            text=f"<b>{grade:.1f}%</b><br><span style='font-size:14px; color:#7F8C8D'>Grade</span>",
            x=0.5, y=0.5,
            font=dict(size=36, color=grade_color, family='Times New Roman'),
            showarrow=False,
            align='center'
        ),
        # Grade description below the chart
        dict(
            text=f"<b>Performance Level:</b> {'Excellent' if grade >= 80 else 'Good' if grade >= 60 else 'Needs Improvement'}",
            x=0.5, y=0.05,
            font=dict(size=14, color=grade_color, family='Times New Roman'),
            showarrow=False,
            align='center'
        )
    ],
    margin=dict(t=100, b=80, l=50, r=50)
)

# Display the chart
fig.show()

print(f"Financial Grade: {grade:.1f}/100")
print(f"Performance Level: {'Excellent' if grade >= 80 else 'Good' if grade >= 60 else 'Needs Improvement'}")
print(f"Hover over the chart to see detailed breakdown!")

Financial Grade: 89.6/100
Performance Level: Excellent
Hover over the chart to see detailed breakdown!


In [52]:
# DATA SETUP

sarah_data = {
    'salary': 85000 / 12, 'freelance': 8000 / 12, 'dividends': 1200 / 12,
    'rent': -1800, 'utilities': -(120 + 45 + 35 + 65 + 35),
    'insurance': -(220 + 95 + 65), 'debt_payments': -(425 + 180 + 295),
    'phone': -85, 'gym': -65, 'subscriptions': -25,
    'groceries': -(110 * 4.33), 'dining_out': -(30 * 4.33),
    'gas_transport': -(45 * 4.33),
}

income_categories = ['Salary', 'Freelance', 'Investment Income']
income_amounts = [sarah_data['salary'], sarah_data['freelance'], sarah_data['dividends']]

expense_categories = [
    'Housing (Rent + Utilities)', 'Insurance', 'Debt Payments',
    'Food (Groceries + Dining)', 'Transportation', 'Other (Phone, Gym, Subscriptions)'
]
expense_amounts = [
    sarah_data['rent'] + sarah_data['utilities'], sarah_data['insurance'],
    sarah_data['debt_payments'], sarah_data['groceries'] + sarah_data['dining_out'],
    sarah_data['gas_transport'], sarah_data['phone'] + sarah_data['gym'] + sarah_data['subscriptions']
]

all_categories = income_categories + expense_categories
all_amounts = income_amounts + expense_amounts

df = pd.DataFrame({
    'Category': all_categories,
    'Amount': all_amounts,
    'Type': ['Income'] * len(income_categories) + ['Expense'] * len(expense_categories),
    'Amount_Abs': [abs(x) for x in all_amounts]
})

total_income = sum(income_amounts)
total_expenses = sum(expense_amounts)
net_cash_flow = total_income + total_expenses

print(f"📊 Sarah's Monthly Summary: Income: ${total_income:,.0f} | Expenses: ${abs(total_expenses):,.0f} | Net: ${net_cash_flow:,.0f}")

# BLOCK 1: INTERACTIVE INCOME VS EXPENSES HORIZONTAL BAR CHART

# Sort by absolute amount for better visual hierarchy
df_sorted = df.sort_values('Amount_Abs', ascending=True)

# Separate income and expenses
income_data = df_sorted[df_sorted['Type'] == 'Income']
expense_data = df_sorted[df_sorted['Type'] == 'Expense']

# Create the figure
fig1 = go.Figure()

# Add income bars
fig1.add_trace(go.Bar(
    name='💰 Income',
    y=income_data['Category'],
    x=income_data['Amount'],
    orientation='h',
    marker=dict(
        color=['#27AE60', '#2ECC71', '#58D68D'][:len(income_data)],
        line=dict(color='white', width=2)
    ),
    hovertemplate='<b>%{y}</b><br>' +
                  'Amount: $%{x:,.0f}<br>' +
                  'Type: Income<br>' +
                  '<extra></extra>',
    text=[f'${x:,.0f}' for x in income_data['Amount']],
    textfont=dict(color='white', size=12, family='Times New Roman')
))

# Add expense bars
fig1.add_trace(go.Bar(
    name='💸 Expenses',
    y=expense_data['Category'],
    x=expense_data['Amount'],
    orientation='h',
    marker=dict(
        color=['#E74C3C', '#EC7063', '#F1948A', '#F5B7B1', '#FADBD8', '#FDEDEC'][:len(expense_data)],
        line=dict(color='white', width=2)
    ),
    hovertemplate='<b>%{y}</b><br>' +
                  'Amount: $%{x:,.0f}<br>' +
                  'Type: Expense<br>' +
                  'Absolute Value: $%{customdata:,.0f}<br>' +
                  '<extra></extra>',
    customdata=[abs(x) for x in expense_data['Amount']],
    text=[f'${abs(x):,.0f}' for x in expense_data['Amount']],
    textfont=dict(color='white', size=12, family='Times New Roman')
))

# Add vertical line at zero
fig1.add_vline(x=0, line=dict(color='#2C3E50', width=3))

# Update layout for beautiful styling
fig1.update_layout(
    title=dict(
        text='💼 Sarah\'s Monthly Cash Flow Analysis<br><sub>Hover over bars for detailed information</sub>',
        font=dict(size=20, family='Times New Roman', color='#2C3E50'),
        x=0.5
    ),
    xaxis=dict(
        title='Monthly Amount ($)',
        titlefont=dict(size=14, family='Times New Roman', color='#2C3E50'),
        tickformat='$,.0f',
        gridcolor='#E5E5E5',
        gridwidth=1,
        zeroline=True,
        zerolinecolor='#2C3E50',
        zerolinewidth=2
    ),
    yaxis=dict(
        titlefont=dict(size=14, family='Times New Roman', color='#2C3E50'),
        tickfont=dict(size=11, family='Times New Roman')
    ),
    plot_bgcolor='white',
    paper_bgcolor='#F8F9FA',
    font=dict(family='Times New Roman'),
    hovermode='y unified',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1,
        font=dict(size=12, family='Times New Roman')
    ),
    height=600,
    annotations=[
        dict(
            x=0.02, y=0.98,
            xref='paper', yref='paper',
            text=f'📊 Summary:<br>Monthly Income: ${total_income:,.0f} | Monthly Expenses: ${abs(total_expenses):,.0f}<br>Net Cash Flow: ${net_cash_flow:,.0f}<br>Savings Rate: {(net_cash_flow/total_income)*100:.1f}%',
            showarrow=False,
            align='left',
            bgcolor='lightblue',
            bordercolor='#3498DB',
            borderwidth=2,
            font=dict(size=10, family='Times New Roman')
        )
    ]
)

fig1.show()

📊 Sarah's Monthly Summary: Income: $7,850 | Expenses: $4,356 | Net: $3,494


In [53]:
#INTERACTIVE HORIZONTAL HISTOGRAM (SORTED)

# Sort all categories by absolute amount (descending)
df_hist = df.sort_values('Amount_Abs', ascending=False)

# Create color mapping
colors = []
for amount_type in df_hist['Type']:
    if amount_type == 'Income':
        colors.append('#27AE60')
    else:
        colors.append('#E74C3C')

fig2 = go.Figure()

fig2.add_trace(go.Bar(
    name='Cash Flow Categories',
    y=df_hist['Category'],
    x=df_hist['Amount_Abs'],
    orientation='h',
    marker=dict(
        color=colors,
        line=dict(color='white', width=2),
        opacity=0.85
    ),
    hovertemplate='<b>%{y}</b><br>' +
                  'Amount: $%{x:,.0f}<br>' +
                  'Type: %{customdata}<br>' +
                  'Percentage of Total: %{text}<br>' +
                  '<extra></extra>',
    customdata=df_hist['Type'],
    text=[f'{(amt/(total_income + abs(total_expenses)))*100:.1f}%' for amt in df_hist['Amount_Abs']],
    texttemplate='💰 $%{x:,.0f}' if df_hist['Type'].iloc[0] == 'Income' else '💸 $%{x:,.0f}',
    textfont=dict(color='white', size=11, family='Times New Roman')
))

fig2.update_layout(
    title=dict(
        text='📈 Cash Flow Categories (Ranked by Amount)<br><sub>Interactive view - hover for details</sub>',
        font=dict(size=20, family='Times New Roman', color='#2C3E50'),
        x=0.5
    ),
    xaxis=dict(
        title='Amount ($)',
        titlefont=dict(size=14, family='Times New Roman', color='#2C3E50'),
        tickformat='$,.0f',
        gridcolor='#E5E5E5',
        gridwidth=1
    ),
    yaxis=dict(
        titlefont=dict(size=14, family='Times New Roman', color='#2C3E50'),
        tickfont=dict(size=11, family='Times New Roman')
    ),
    plot_bgcolor='white',
    paper_bgcolor='#F8F9FA',
    font=dict(family='Times New Roman'),
    showlegend=False,
    height=600,
    hovermode='y'
)

fig2.show()

In [54]:
#INTERACTIVE SAVINGS PROJECTION WITH GOAL TRACKING

house_goal = 50000
months_to_goal = house_goal / net_cash_flow if net_cash_flow > 0 else float('inf')

# Extended projection
months = list(range(1, 25))
cumulative_savings = [net_cash_flow * month for month in months]

fig3 = go.Figure()

# Add savings projection line
fig3.add_trace(go.Scatter(
    x=months,
    y=cumulative_savings,
    mode='lines+markers',
    name='💰 Projected Savings',
    line=dict(color='#3498DB', width=4),
    marker=dict(size=8, color='white', line=dict(color='#3498DB', width=2)),
    fill='tozeroy',
    fillcolor='rgba(52, 152, 219, 0.2)',
    hovertemplate='<b>Month %{x}</b><br>' +
                  'Cumulative Savings: $%{y:,.0f}<br>' +
                  'Monthly Addition: $' + f'{net_cash_flow:,.0f}<br>' +
                  '<extra></extra>'
))

# Add goal line
fig3.add_hline(
    y=house_goal,
    line=dict(color='#E74C3C', width=3, dash='dash'),
    annotation_text=f'🏠 House Down Payment Goal: ${house_goal:,.0f}',
    annotation_position='top right'
)

# Add goal achievement point
if months_to_goal <= 24:
    goal_month = int(months_to_goal)
    goal_savings = net_cash_flow * goal_month
    fig3.add_trace(go.Scatter(
        x=[goal_month],
        y=[goal_savings],
        mode='markers',
        name='🎯 Goal Achieved!',
        marker=dict(size=15, color='#E74C3C', symbol='star'),
        hovertemplate='<b>Goal Reached!</b><br>' +
                      'Month: %{x}<br>' +
                      'Amount: $%{y:,.0f}<br>' +
                      '<extra></extra>'
    ))

# Add milestone markers
milestone_months = [6, 12, 18]
milestone_savings = [net_cash_flow * m for m in milestone_months if m <= 24]
milestone_months_filtered = [m for m in milestone_months if m <= 24]

if milestone_months_filtered:
    fig3.add_trace(go.Scatter(
        x=milestone_months_filtered,
        y=milestone_savings,
        mode='markers',
        name='📍 Milestones',
        marker=dict(size=10, color='#F39C12', symbol='diamond'),
        hovertemplate='<b>Milestone - Month %{x}</b><br>' +
                      'Savings: $%{y:,.0f}<br>' +
                      '<extra></extra>'
    ))

fig3.update_layout(
    title=dict(
        text='🚀 Sarah\'s Interactive Savings Journey<br><sub>Track progress toward home ownership goal</sub>',
        font=dict(size=20, family='Times New Roman', color='#2C3E50'),
        x=0.5
    ),
    xaxis=dict(
        title='Months',
        titlefont=dict(size=14, family='Times New Roman', color='#2C3E50'),
        tickfont=dict(size=11, family='Times New Roman'),
        gridcolor='#E5E5E5',
        gridwidth=1
    ),
    yaxis=dict(
        title='Cumulative Savings ($)',
        titlefont=dict(size=14, family='Times New Roman', color='#2C3E50'),
        tickformat='$,.0f',
        tickfont=dict(size=11, family='Times New Roman'),
        gridcolor='#E5E5E5',
        gridwidth=1
    ),
    plot_bgcolor='white',
    paper_bgcolor='#F8F9FA',
    font=dict(family='Times New Roman'),
    hovermode='x unified',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1,
        font=dict(size=12, family='Times New Roman')
    ),
    height=600
)

fig3.show()

print(f"🎯 Interactive Analysis Complete!")
print(f"Goal Achievement: {months_to_goal:.1f} months to reach ${house_goal:,} down payment!")

🎯 Interactive Analysis Complete!
Goal Achievement: 14.3 months to reach $50,000 down payment!


In [75]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Financial data
current_debt = 900
monthly_income = 7850
freelance_income = 667
house_goal = 50000

# Soothing color palette
colors = {
    'primary': '#6B73FF',      # Soft blue
    'secondary': '#9F7AEA',    # Soft purple
    'success': '#48BB78',      # Soft green
    'warning': '#ED8936',      # Soft orange
    'danger': '#F56565',       # Soft red
    'neutral': '#A0AEC0',      # Soft gray
    'light': '#F7FAFC',        # Very light gray
    'accent': '#38B2AC'        # Soft teal
}

# Create simplified dashboard with 5 key visuals
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=(
        'Your Top 3 Priorities',
        'Debt Payoff Comparison',
        'Emergency Fund Progress',
        'Income Growth Plan',
        'Monthly Budget Optimization',
        ''
    ),
    specs=[
        [{"type": "bar"}, {"type": "scatter"}, {"type": "scatter"}],
        [{"type": "bar"}, {"type": "bar"}, None]
    ],
    vertical_spacing=0.15,
    horizontal_spacing=0.10
)

# 1. TOP 3 PRIORITIES (Row 1, Col 1)
priorities = ['Build Emergency Fund', 'Optimize Debt Payments', 'Plan Housing Costs']
urgency = [10, 8, 7]
priority_colors = [colors['danger'], colors['warning'], colors['primary']]

fig.add_trace(go.Bar(
    x=priorities,
    y=urgency,
    marker_color=priority_colors,
    text=[f'{u}/10' for u in urgency],
    textposition='outside',
    textfont=dict(size=12, color='#2D3748'),
    hovertemplate='<b>%{x}</b><br>Priority Level: %{y}/10<extra></extra>',
    showlegend=False
), row=1, col=1)

# 2. DEBT PAYOFF COMPARISON (Row 1, Col 2)
months = np.array([0, 6, 12, 18, 24])
minimum_debt = np.array([10800, 9400, 7800, 6000, 4000])
avalanche_debt = np.array([10800, 8000, 4500, 1500, 0])

fig.add_trace(go.Scatter(
    x=months,
    y=minimum_debt,
    mode='lines+markers',
    name='Minimum Payments',
    line=dict(color=colors['neutral'], width=3),
    marker=dict(size=8),
    hovertemplate='Month %{x}<br>Debt Remaining: $%{y:,.0f}<extra></extra>'
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=months,
    y=avalanche_debt,
    mode='lines+markers',
    name='Debt Avalanche Method',
    line=dict(color=colors['success'], width=3),
    marker=dict(size=8),
    fill='tonexty',
    fillcolor='rgba(72, 187, 120, 0.1)',
    hovertemplate='Month %{x}<br>Debt Remaining: $%{y:,.0f}<br>Debt-free faster!<extra></extra>'
), row=1, col=2)

# 3. EMERGENCY FUND PROGRESS (Row 1, Col 3)
emergency_months = [0, 3, 6, 9]
fund_balance = [0, 10500, 21000, 26000]

fig.add_trace(go.Scatter(
    x=emergency_months,
    y=fund_balance,
    mode='lines+markers',
    line=dict(color=colors['accent'], width=4),
    marker=dict(size=12, color=colors['accent']),
    fill='tozeroy',
    fillcolor='rgba(56, 178, 172, 0.1)',
    hovertemplate='Month %{x}<br>Emergency Fund: $%{y:,.0f}<extra></extra>',
    showlegend=False
), row=1, col=3)

# Add target line for emergency fund
fig.add_hline(y=26000, line_dash="dash", line_color=colors['success'],
              annotation_text="6-Month Target", row=1, col=3)

# 4. INCOME GROWTH PLAN (Row 2, Col 1) - Descending order with purple, blue, pink
income_sources = ['New Clients', 'Rate Increase', 'Current Freelance']
amounts = [300, 200, 667]
source_colors = ['#9F7AEA', '#6B73FF', '#FF69B4']  # Purple, blue, pink

fig.add_trace(go.Bar(
    x=income_sources,
    y=amounts,
    marker_color=source_colors,
    text=[f'${a}' for a in amounts],
    textposition='outside',
    textfont=dict(size=12, color='#2D3748'),
    hovertemplate='<b>%{x}</b><br>Monthly Income: $%{y}<extra></extra>',
    showlegend=False
), row=2, col=1)

# 5. MONTHLY BUDGET OPTIMIZATION (Row 2, Col 2)
categories = ['Food', 'Subscriptions', 'Utilities']
current_spending = [606, 175, 300]
optimized_spending = [500, 100, 250]

fig.add_trace(go.Bar(
    name='Current',
    x=categories,
    y=current_spending,
    marker_color=colors['danger'],
    opacity=0.7,
    hovertemplate='<b>%{x}</b><br>Current: $%{y}<extra></extra>'
), row=2, col=2)

fig.add_trace(go.Bar(
    name='Optimized',
    x=categories,
    y=optimized_spending,
    marker_color=colors['success'],
    opacity=0.8,
    hovertemplate='<b>%{x}</b><br>Optimized: $%{y}<extra></extra>'
), row=2, col=2)

# Update layout with clean, calming design
fig.update_layout(
    title=dict(
        text='Your Simplified Financial Action Plan',
        font=dict(size=24, family='Times New Roman', color='#2D3748'),
        x=0.5,
        pad=dict(t=20)
    ),
    height=800,
    plot_bgcolor='white',
    paper_bgcolor=colors['light'],
    font=dict(family='Times New Roman', size=11, color='#4A5568'),
    showlegend=True,
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=-0.1,
        xanchor='center',
        x=0.5,
        bgcolor='rgba(255,255,255,0.8)',
        bordercolor=colors['neutral'],
        borderwidth=1
    ),
    margin=dict(t=80, b=60, l=60, r=60)
)

# Clean up axes with soft styling - update loop to skip empty subplot
for i in range(1, 6):  # Changed from 7 to 6 since we have 5 charts now
    if i <= 3:
        row = 1
        col = i
    else:
        row = 2
        col = i - 3

    fig.update_xaxes(
        gridcolor='rgba(160, 174, 192, 0.2)',
        zerolinecolor='rgba(160, 174, 192, 0.3)',
        tickfont=dict(color='#4A5568'),
        row=row, col=col
    )

    fig.update_yaxes(
        gridcolor='rgba(160, 174, 192, 0.2)',
        zerolinecolor='rgba(160, 174, 192, 0.3)',
        tickfont=dict(color='#4A5568'),
        tickformat='$,.0f' if i in [2, 3, 4, 5] else None,
        row=row, col=col
    )

fig.show()

print("Simplified Financial Dashboard Complete!")
print("Focus on these 5 key areas for maximum financial impact.")

Simplified Financial Dashboard Complete!
Focus on these 5 key areas for maximum financial impact.
